<a href="https://colab.research.google.com/github/ianellisjones/usn/blob/main/Big_Deck_Fleet_Globe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌐 US NAVY BIG DECK FLEET TRACKER
### Interactive Globe Visualization

**Palantir meets War Games aesthetic** - Track the real-time deployment status of US Navy Aircraft Carriers (CVN) and Amphibious Assault Ships (LHA/LHD).

---
**Instructions:**
1. Run all cells in order
2. The app will generate in the output below
3. A shareable HTML file will be saved that you can download

*Data Source: USCarriers.net (with permission)*

In [ ]:
# Cell 1: Install Dependencies
!pip install -q requests beautifulsoup4 pandas plotly

In [ ]:
"""
US NAVY BIG DECK FLEET TRACKER - GLOBE EDITION
Version: 5.0.0

A comprehensive fleet tracker with:
- Improved scraping with robust error handling
- Interactive 3D globe visualization
- War Games / Palantir aesthetic (dark theme, neon accents)
- Click-to-view ship details panel
- Shareable HTML output

Data Source: USCarriers.net (with permission)
"""

import re
import json
from datetime import datetime
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass, asdict
from IPython.display import HTML, display

import requests
from bs4 import BeautifulSoup
import pandas as pd

# ==============================================================================
# CONFIGURATION
# ==============================================================================

USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'

# Ship Database with hull numbers and names
SHIP_DATABASE = {
    # Aircraft Carriers (CVN)
    "CVN68": {"name": "USS Nimitz", "class": "Nimitz", "type": "CVN", "url": "http://uscarriers.net/cvn68history.htm"},
    "CVN69": {"name": "USS Dwight D. Eisenhower", "class": "Nimitz", "type": "CVN", "url": "http://uscarriers.net/cvn69history.htm"},
    "CVN70": {"name": "USS Carl Vinson", "class": "Nimitz", "type": "CVN", "url": "http://uscarriers.net/cvn70history.htm"},
    "CVN71": {"name": "USS Theodore Roosevelt", "class": "Nimitz", "type": "CVN", "url": "http://uscarriers.net/cvn71history.htm"},
    "CVN72": {"name": "USS Abraham Lincoln", "class": "Nimitz", "type": "CVN", "url": "http://uscarriers.net/cvn72history.htm"},
    "CVN73": {"name": "USS George Washington", "class": "Nimitz", "type": "CVN", "url": "http://uscarriers.net/cvn73history.htm"},
    "CVN74": {"name": "USS John C. Stennis", "class": "Nimitz", "type": "CVN", "url": "http://uscarriers.net/cvn74history.htm"},
    "CVN75": {"name": "USS Harry S. Truman", "class": "Nimitz", "type": "CVN", "url": "http://uscarriers.net/cvn75history.htm"},
    "CVN76": {"name": "USS Ronald Reagan", "class": "Nimitz", "type": "CVN", "url": "http://uscarriers.net/cvn76history.htm"},
    "CVN77": {"name": "USS George H.W. Bush", "class": "Nimitz", "type": "CVN", "url": "http://uscarriers.net/cvn77history.htm"},
    "CVN78": {"name": "USS Gerald R. Ford", "class": "Ford", "type": "CVN", "url": "http://uscarriers.net/cvn78history.htm"},
    # Amphibious Assault Ships (LHA/LHD)
    "LHD1": {"name": "USS Wasp", "class": "Wasp", "type": "LHD", "url": "http://uscarriers.net/lhd1history.htm"},
    "LHD2": {"name": "USS Essex", "class": "Wasp", "type": "LHD", "url": "http://uscarriers.net/lhd2history.htm"},
    "LHD3": {"name": "USS Kearsarge", "class": "Wasp", "type": "LHD", "url": "http://uscarriers.net/lhd3history.htm"},
    "LHD4": {"name": "USS Boxer", "class": "Wasp", "type": "LHD", "url": "http://uscarriers.net/lhd4history.htm"},
    "LHD5": {"name": "USS Bataan", "class": "Wasp", "type": "LHD", "url": "http://uscarriers.net/lhd5history.htm"},
    "LHD7": {"name": "USS Iwo Jima", "class": "Wasp", "type": "LHD", "url": "http://uscarriers.net/lhd7history.htm"},
    "LHD8": {"name": "USS Makin Island", "class": "Wasp", "type": "LHD", "url": "http://uscarriers.net/lhd8history.htm"},
    "LHA6": {"name": "USS America", "class": "America", "type": "LHA", "url": "http://uscarriers.net/lha6history.htm"},
    "LHA7": {"name": "USS Tripoli", "class": "America", "type": "LHA", "url": "http://uscarriers.net/lha7history.htm"},
}

# Comprehensive coordinate database with accurate lat/lon
LOCATION_COORDS = {
    # === US PORTS / SHIPYARDS ===
    "Norfolk / Portsmouth": {"lat": 36.9473, "lon": -76.3134, "region": "CONUS"},
    "San Diego": {"lat": 32.7157, "lon": -117.1611, "region": "CONUS"},
    "Bremerton / Kitsap": {"lat": 47.5673, "lon": -122.6329, "region": "CONUS"},
    "Newport News": {"lat": 36.9788, "lon": -76.4280, "region": "CONUS"},
    "Pearl Harbor": {"lat": 21.3545, "lon": -157.9698, "region": "PACIFIC"},
    "Mayport": {"lat": 30.3918, "lon": -81.4285, "region": "CONUS"},
    "Everett": {"lat": 47.9790, "lon": -122.2021, "region": "CONUS"},
    "Pascagoula": {"lat": 30.3658, "lon": -88.5561, "region": "CONUS"},
    
    # === FORWARD DEPLOYED / FOREIGN PORTS ===
    "Yokosuka": {"lat": 35.2831, "lon": 139.6703, "region": "WESTPAC"},
    "Sasebo": {"lat": 33.1595, "lon": 129.7235, "region": "WESTPAC"},
    "Guam": {"lat": 13.4443, "lon": 144.7937, "region": "WESTPAC"},
    "Singapore": {"lat": 1.2655, "lon": 103.8200, "region": "INDOPAC"},
    "Bahrain": {"lat": 26.2235, "lon": 50.5876, "region": "CENTCOM"},
    "Dubai": {"lat": 25.2582, "lon": 55.3047, "region": "CENTCOM"},
    "Busan": {"lat": 35.1028, "lon": 129.0403, "region": "WESTPAC"},
    "Philippines": {"lat": 14.5995, "lon": 120.9842, "region": "WESTPAC"},
    "Malaysia": {"lat": 3.1390, "lon": 101.6869, "region": "INDOPAC"},
    "Okinawa": {"lat": 26.3344, "lon": 127.8056, "region": "WESTPAC"},
    
    # === STRATEGIC REGIONS / SEAS ===
    "South China Sea": {"lat": 12.0000, "lon": 114.0000, "region": "WESTPAC"},
    "Western Pacific (WESTPAC)": {"lat": 15.0000, "lon": 135.0000, "region": "WESTPAC"},
    "Philippine Sea": {"lat": 20.0000, "lon": 130.0000, "region": "WESTPAC"},
    "Red Sea": {"lat": 20.0000, "lon": 38.0000, "region": "CENTCOM"},
    "Persian Gulf": {"lat": 27.0000, "lon": 51.0000, "region": "CENTCOM"},
    "Gulf of Oman": {"lat": 24.5000, "lon": 58.5000, "region": "CENTCOM"},
    "Gulf of Aden": {"lat": 12.5000, "lon": 47.0000, "region": "CENTCOM"},
    "Arabian Sea": {"lat": 15.0000, "lon": 65.0000, "region": "CENTCOM"},
    "Mediterranean": {"lat": 35.0000, "lon": 18.0000, "region": "EUCOM"},
    "Caribbean Sea": {"lat": 15.5000, "lon": -73.0000, "region": "SOUTHCOM"},
    "North Sea": {"lat": 56.0000, "lon": 3.0000, "region": "EUCOM"},
    "Norwegian Sea": {"lat": 68.0000, "lon": 5.0000, "region": "EUCOM"},
    "Strait of Gibraltar": {"lat": 35.9500, "lon": -5.6000, "region": "EUCOM"},
    "Suez Canal": {"lat": 30.6000, "lon": 32.3300, "region": "CENTCOM"},
    "Bab el-Mandeb": {"lat": 12.5833, "lon": 43.3333, "region": "CENTCOM"},
    "Sea of Japan": {"lat": 40.0000, "lon": 135.0000, "region": "WESTPAC"},
    
    # === OCEANS (Broad Areas) ===
    "Atlantic Ocean": {"lat": 30.0000, "lon": -45.0000, "region": "ATLANTIC"},
    "Pacific Ocean": {"lat": 20.0000, "lon": -150.0000, "region": "PACIFIC"},
    "Indian Ocean": {"lat": -5.0000, "lon": 75.0000, "region": "INDOPAC"},
    
    # === FALLBACK ===
    "Underway / Unknown": {"lat": 0.0000, "lon": 0.0000, "region": "UNKNOWN"},
}


@dataclass
class ShipStatus:
    """Data class representing a ship's current status."""
    hull: str
    name: str
    ship_class: str
    ship_type: str
    location: str
    lat: float
    lon: float
    region: str
    date: str
    status: str
    source_url: str


# ==============================================================================
# SCRAPER ENGINE
# ==============================================================================

def fetch_history_text(url: str, char_limit: int = 50000) -> str:
    """
    Fetches raw HTML content, strips tags, returns the tail of the text.
    """
    try:
        response = requests.get(url, headers={'User-Agent': USER_AGENT}, timeout=20)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        full_text = soup.get_text(separator='\n')
        lines = [line.strip() for line in full_text.split('\n') if line.strip()]
        clean_text = '\n'.join(lines)
        return clean_text[-char_limit:] if len(clean_text) > char_limit else clean_text
    except requests.RequestException as e:
        return f"ERROR: {str(e)}"


def parse_status_entry(text_block: str) -> Tuple[str, str]:
    """
    Parses text block to find the most recent status entry.
    Uses contextual year tracking for entries lacking explicit years.
    """
    lines = text_block.split('\n')
    
    # Determine context year
    current_year = "Unknown"
    years_found = re.findall(r'(202[3-7])', text_block)
    if years_found:
        priority_years = [y for y in years_found if y in ['2024', '2025', '2026']]
        current_year = priority_years[-1] if priority_years else years_found[-1]
    
    processed_lines = []
    running_year = current_year
    
    for line in lines:
        year_match = re.search(r'^202[3-7]', line)
        if year_match:
            running_year = year_match.group(0)
        processed_lines.append({'text': line, 'year': running_year})
    
    # Status keywords to look for
    keywords = [
        "moored", "anchored", "underway", "arrived", "departed",
        "transited", "operations", "returned", "participated", "conducted",
        "moved to", "visited", "pulled into", "sea trials", "flight deck",
        "undocked", "homeport"
    ]
    allowed_years = ["2024", "2025", "2026", "2027"]
    
    # Bottom-up search for latest entry
    for entry in reversed(processed_lines):
        text_lower = entry['text'].lower()
        year = entry['year']
        if year in allowed_years and any(k in text_lower for k in keywords):
            if text_lower.strip().startswith("from ") and " - " in text_lower:
                continue
            return year, entry['text']
    
    return current_year, "No recent status found."


def categorize_location(text: str) -> str:
    """
    Maps keywords in status text to high-level location tags.
    Includes departure overrides to show current ocean rather than origin port.
    """
    text_lower = text.lower()
    
    # Movement overrides - prioritize destination over departure point
    departure_overrides = {
        "departed san diego": "Pacific Ocean",
        "departed norfolk": "Atlantic Ocean",
        "departed pearl harbor": "Pacific Ocean",
        "departed mayport": "Atlantic Ocean",
        "departed bremerton": "Pacific Ocean",
        "departed yokosuka": "Western Pacific (WESTPAC)",
        "departed sasebo": "Western Pacific (WESTPAC)",
    }
    
    for phrase, location in departure_overrides.items():
        if phrase in text_lower:
            return location
    
    # Location mapping - ordered by specificity (most specific first)
    location_map = {
        # Specific Regions
        "South China Sea": ["south china sea", "spratly islands", "spratly", "luzon"],
        "Western Pacific (WESTPAC)": ["san bernardino strait", "western pacific", "westpac"],
        "Philippine Sea": ["philippine sea"],
        "Red Sea": ["red sea"],
        "Persian Gulf": ["persian gulf", "arabian gulf"],
        "Gulf of Oman": ["gulf of oman"],
        "Gulf of Aden": ["gulf of aden"],
        "Arabian Sea": ["arabian sea"],
        "Mediterranean": ["mediterranean", "med sea"],
        "Caribbean Sea": ["caribbean", "st. croix", "trinidad", "tobago", "puerto rico", "virgin islands"],
        "North Sea": ["north sea"],
        "Norwegian Sea": ["norwegian sea"],
        "Strait of Gibraltar": ["gibraltar"],
        "Suez Canal": ["suez"],
        "Bab el-Mandeb": ["bab el-mandeb"],
        "Sea of Japan": ["sea of japan"],
        
        # Ports / Shipyards
        "Norfolk / Portsmouth": ["norfolk", "portsmouth", "virginia beach", "naval station norfolk", "pier 1", "pier 11", "pier 12", "pier 14"],
        "San Diego": ["san diego", "north island", "camp pendleton", "nassco san diego"],
        "Bremerton / Kitsap": ["bremerton", "kitsap", "psns"],
        "Newport News": ["newport news", "huntington ingalls", "outfitting berth"],
        "Yokosuka": ["yokosuka"],
        "Pearl Harbor": ["pearl harbor"],
        "Mayport": ["mayport"],
        "Everett": ["everett"],
        "Singapore": ["singapore", "changi"],
        "Bahrain": ["bahrain", "manama"],
        "Dubai": ["dubai", "jebel ali"],
        "Busan": ["busan"],
        "Guam": ["guam", "apra"],
        "Sasebo": ["sasebo", "juliet basin"],
        "Malaysia": ["malaysia", "klang"],
        "Philippines": ["philippines", "manila", "subic"],
        "Pascagoula": ["pascagoula", "ingalls"],
        "Okinawa": ["okinawa", "white beach"],
        
        # Broad Oceans (Prioritized Last)
        "Atlantic Ocean": ["atlantic"],
        "Pacific Ocean": ["pacific"],
        "Indian Ocean": ["indian ocean"],
    }
    
    for label, keywords in location_map.items():
        if any(k in text_lower for k in keywords):
            return label
    
    return "Underway / Unknown"


def extract_date(text: str) -> str:
    """Extracts the last specific date mentioned in the text."""
    pattern = r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\.?\s+\d{1,2}'
    matches = re.findall(pattern, text, re.IGNORECASE)
    return matches[-1] if matches else "Date Unspecified"


def scrape_fleet() -> List[ShipStatus]:
    """Scrapes all ships and returns list of ShipStatus objects."""
    results = []
    total = len(SHIP_DATABASE)
    
    print("\n" + "="*70)
    print("  US NAVY BIG DECK FLEET TRACKER - SCANNING")
    print("="*70 + "\n")
    
    for i, (hull, ship_info) in enumerate(SHIP_DATABASE.items(), 1):
        print(f"  [{i:2}/{total}] Scanning {hull} - {ship_info['name']}...", end=" ")
        
        raw_text = fetch_history_text(ship_info['url'])
        
        if "ERROR" in raw_text:
            print("❌ FAILED")
            continue
        
        year, status = parse_status_entry(raw_text)
        location = categorize_location(status)
        date_str = extract_date(status)
        
        if date_str == "Date Unspecified":
            date_str = year
        
        # Get coordinates
        coords = LOCATION_COORDS.get(location, LOCATION_COORDS["Underway / Unknown"])
        
        ship_status = ShipStatus(
            hull=hull,
            name=ship_info['name'],
            ship_class=ship_info['class'],
            ship_type=ship_info['type'],
            location=location,
            lat=coords['lat'],
            lon=coords['lon'],
            region=coords['region'],
            date=date_str,
            status=status,
            source_url=ship_info['url']
        )
        results.append(ship_status)
        print(f"✓ {location}")
    
    print("\n" + "="*70)
    print(f"  SCAN COMPLETE: {len(results)} ships tracked")
    print("="*70 + "\n")
    
    return results


# ==============================================================================
# VISUALIZATION ENGINE
# ==============================================================================

def generate_globe_html(ships: List[ShipStatus]) -> str:
    """
    Generates a complete HTML page with an interactive 3D globe.
    War Games / Palantir aesthetic: dark theme, neon accents, terminal style.
    """
    
    # Convert ships to JSON for JavaScript
    ships_data = [asdict(s) for s in ships]
    ships_json = json.dumps(ships_data)
    
    # Count ships by type
    cvn_count = len([s for s in ships if s.ship_type == "CVN"])
    lha_lhd_count = len([s for s in ships if s.ship_type in ["LHA", "LHD"]])
    
    # Get current timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S UTC")
    
    html = f'''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>US NAVY BIG DECK FLEET TRACKER</title>
    <script src="https://cdn.plot.ly/plotly-2.27.0.min.js"></script>
    <link href="https://fonts.googleapis.com/css2?family=Share+Tech+Mono&family=Orbitron:wght@400;700&display=swap" rel="stylesheet">
    <style>
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}
        
        body {{
            background: #0a0a0f;
            color: #00ff88;
            font-family: 'Share Tech Mono', monospace;
            overflow: hidden;
            height: 100vh;
        }}
        
        /* Scanline effect */
        body::before {{
            content: "";
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            background: repeating-linear-gradient(
                0deg,
                rgba(0, 0, 0, 0.15),
                rgba(0, 0, 0, 0.15) 1px,
                transparent 1px,
                transparent 2px
            );
            pointer-events: none;
            z-index: 1000;
        }}
        
        .container {{
            display: grid;
            grid-template-columns: 1fr 380px;
            grid-template-rows: auto 1fr;
            height: 100vh;
            gap: 0;
        }}
        
        /* Header */
        .header {{
            grid-column: 1 / -1;
            background: linear-gradient(180deg, #0d1117 0%, #0a0a0f 100%);
            border-bottom: 2px solid #00ff88;
            padding: 12px 24px;
            display: flex;
            justify-content: space-between;
            align-items: center;
            box-shadow: 0 4px 20px rgba(0, 255, 136, 0.15);
        }}
        
        .header-left {{
            display: flex;
            align-items: center;
            gap: 20px;
        }}
        
        .logo {{
            font-family: 'Orbitron', sans-serif;
            font-size: 24px;
            font-weight: 700;
            color: #00ff88;
            text-shadow: 0 0 10px rgba(0, 255, 136, 0.5);
            letter-spacing: 2px;
        }}
        
        .logo-sub {{
            font-size: 11px;
            color: #00aa55;
            letter-spacing: 4px;
            margin-top: 2px;
        }}
        
        .status-bar {{
            display: flex;
            gap: 30px;
        }}
        
        .stat {{
            text-align: center;
        }}
        
        .stat-value {{
            font-family: 'Orbitron', sans-serif;
            font-size: 28px;
            font-weight: 700;
            color: #00ffff;
            text-shadow: 0 0 15px rgba(0, 255, 255, 0.5);
        }}
        
        .stat-value.cvn {{
            color: #ff6b6b;
            text-shadow: 0 0 15px rgba(255, 107, 107, 0.5);
        }}
        
        .stat-value.amphib {{
            color: #4ecdc4;
            text-shadow: 0 0 15px rgba(78, 205, 196, 0.5);
        }}
        
        .stat-label {{
            font-size: 10px;
            color: #666;
            letter-spacing: 2px;
            text-transform: uppercase;
        }}
        
        .timestamp {{
            font-size: 11px;
            color: #444;
        }}
        
        .timestamp span {{
            color: #00ff88;
        }}
        
        /* Globe */
        .globe-container {{
            background: #0a0a0f;
            position: relative;
        }}
        
        #globe {{
            width: 100%;
            height: 100%;
        }}
        
        /* Side Panel */
        .side-panel {{
            background: linear-gradient(180deg, #0d1117 0%, #0a0a0f 100%);
            border-left: 2px solid #00ff88;
            display: flex;
            flex-direction: column;
            overflow: hidden;
        }}
        
        .panel-header {{
            padding: 15px 20px;
            border-bottom: 1px solid #1a1a2e;
            background: rgba(0, 255, 136, 0.05);
        }}
        
        .panel-title {{
            font-family: 'Orbitron', sans-serif;
            font-size: 14px;
            color: #00ff88;
            letter-spacing: 3px;
        }}
        
        .ship-list {{
            flex: 1;
            overflow-y: auto;
            padding: 10px;
        }}
        
        .ship-list::-webkit-scrollbar {{
            width: 6px;
        }}
        
        .ship-list::-webkit-scrollbar-track {{
            background: #0a0a0f;
        }}
        
        .ship-list::-webkit-scrollbar-thumb {{
            background: #00ff88;
            border-radius: 3px;
        }}
        
        .ship-card {{
            background: rgba(26, 26, 46, 0.6);
            border: 1px solid #1a1a2e;
            border-radius: 4px;
            margin-bottom: 8px;
            padding: 12px;
            cursor: pointer;
            transition: all 0.2s ease;
        }}
        
        .ship-card:hover {{
            border-color: #00ff88;
            background: rgba(0, 255, 136, 0.08);
            box-shadow: 0 0 15px rgba(0, 255, 136, 0.1);
        }}
        
        .ship-card.active {{
            border-color: #00ffff;
            background: rgba(0, 255, 255, 0.1);
            box-shadow: 0 0 20px rgba(0, 255, 255, 0.2);
        }}
        
        .ship-card.cvn {{
            border-left: 3px solid #ff6b6b;
        }}
        
        .ship-card.amphib {{
            border-left: 3px solid #4ecdc4;
        }}
        
        .ship-header {{
            display: flex;
            justify-content: space-between;
            align-items: flex-start;
            margin-bottom: 8px;
        }}
        
        .ship-hull {{
            font-family: 'Orbitron', sans-serif;
            font-size: 14px;
            font-weight: 700;
            color: #fff;
        }}
        
        .ship-hull.cvn {{
            color: #ff6b6b;
        }}
        
        .ship-hull.amphib {{
            color: #4ecdc4;
        }}
        
        .ship-type {{
            font-size: 9px;
            padding: 2px 6px;
            border-radius: 2px;
            background: rgba(0, 255, 136, 0.2);
            color: #00ff88;
            letter-spacing: 1px;
        }}
        
        .ship-name {{
            font-size: 12px;
            color: #888;
            margin-bottom: 6px;
        }}
        
        .ship-location {{
            font-size: 11px;
            color: #00ff88;
            display: flex;
            align-items: center;
            gap: 6px;
        }}
        
        .ship-location::before {{
            content: "▸";
            color: #00aa55;
        }}
        
        .ship-date {{
            font-size: 10px;
            color: #555;
            margin-top: 4px;
        }}
        
        /* Detail Panel */
        .detail-panel {{
            background: rgba(0, 255, 136, 0.03);
            border-top: 1px solid #1a1a2e;
            padding: 15px;
            max-height: 250px;
            overflow-y: auto;
        }}
        
        .detail-panel.hidden {{
            display: none;
        }}
        
        .detail-header {{
            display: flex;
            justify-content: space-between;
            align-items: center;
            margin-bottom: 12px;
        }}
        
        .detail-title {{
            font-family: 'Orbitron', sans-serif;
            font-size: 16px;
            color: #00ffff;
        }}
        
        .detail-close {{
            cursor: pointer;
            color: #666;
            font-size: 18px;
        }}
        
        .detail-close:hover {{
            color: #ff6b6b;
        }}
        
        .detail-row {{
            display: flex;
            margin-bottom: 8px;
        }}
        
        .detail-label {{
            width: 80px;
            font-size: 10px;
            color: #555;
            text-transform: uppercase;
            letter-spacing: 1px;
        }}
        
        .detail-value {{
            flex: 1;
            font-size: 12px;
            color: #aaa;
        }}
        
        .detail-status {{
            font-size: 11px;
            color: #00ff88;
            line-height: 1.5;
            padding: 10px;
            background: rgba(0, 0, 0, 0.3);
            border-radius: 4px;
            margin-top: 10px;
        }}
        
        /* Region filter */
        .filter-bar {{
            padding: 10px 15px;
            border-bottom: 1px solid #1a1a2e;
            display: flex;
            gap: 8px;
            flex-wrap: wrap;
        }}
        
        .filter-btn {{
            font-family: 'Share Tech Mono', monospace;
            font-size: 10px;
            padding: 4px 10px;
            background: transparent;
            border: 1px solid #333;
            color: #666;
            cursor: pointer;
            border-radius: 2px;
            transition: all 0.2s;
        }}
        
        .filter-btn:hover {{
            border-color: #00ff88;
            color: #00ff88;
        }}
        
        .filter-btn.active {{
            background: rgba(0, 255, 136, 0.2);
            border-color: #00ff88;
            color: #00ff88;
        }}
        
        /* Attribution */
        .attribution {{
            padding: 8px 15px;
            border-top: 1px solid #1a1a2e;
            font-size: 9px;
            color: #333;
            text-align: center;
        }}
        
        .attribution a {{
            color: #444;
        }}
    </style>
</head>
<body>
    <div class="container">
        <header class="header">
            <div class="header-left">
                <div>
                    <div class="logo">FLEET TRACKER</div>
                    <div class="logo-sub">US NAVY BIG DECK OPERATIONS</div>
                </div>
            </div>
            <div class="status-bar">
                <div class="stat">
                    <div class="stat-value">{len(ships)}</div>
                    <div class="stat-label">Total Ships</div>
                </div>
                <div class="stat">
                    <div class="stat-value cvn">{cvn_count}</div>
                    <div class="stat-label">Carriers</div>
                </div>
                <div class="stat">
                    <div class="stat-value amphib">{lha_lhd_count}</div>
                    <div class="stat-label">Amphibs</div>
                </div>
            </div>
            <div class="timestamp">
                LAST UPDATE: <span>{timestamp}</span>
            </div>
        </header>
        
        <div class="globe-container">
            <div id="globe"></div>
        </div>
        
        <div class="side-panel">
            <div class="panel-header">
                <div class="panel-title">▸ FLEET STATUS</div>
            </div>
            <div class="filter-bar">
                <button class="filter-btn active" data-filter="all">ALL</button>
                <button class="filter-btn" data-filter="CVN">CVN</button>
                <button class="filter-btn" data-filter="LHA">LHA</button>
                <button class="filter-btn" data-filter="LHD">LHD</button>
            </div>
            <div class="ship-list" id="shipList"></div>
            <div class="detail-panel hidden" id="detailPanel">
                <div class="detail-header">
                    <div class="detail-title" id="detailTitle">-</div>
                    <span class="detail-close" onclick="closeDetail()">✕</span>
                </div>
                <div class="detail-row">
                    <div class="detail-label">Class</div>
                    <div class="detail-value" id="detailClass">-</div>
                </div>
                <div class="detail-row">
                    <div class="detail-label">Location</div>
                    <div class="detail-value" id="detailLocation">-</div>
                </div>
                <div class="detail-row">
                    <div class="detail-label">Region</div>
                    <div class="detail-value" id="detailRegion">-</div>
                </div>
                <div class="detail-row">
                    <div class="detail-label">As Of</div>
                    <div class="detail-value" id="detailDate">-</div>
                </div>
                <div class="detail-status" id="detailStatus">-</div>
            </div>
            <div class="attribution">
                Data: <a href="http://uscarriers.net" target="_blank">USCarriers.net</a>
            </div>
        </div>
    </div>
    
    <script>
        // Ship data from Python
        const shipsData = {ships_json};
        
        let activeFilter = 'all';
        let selectedShip = null;
        
        // Initialize globe
        function initGlobe() {{
            const cvnShips = shipsData.filter(s => s.ship_type === 'CVN');
            const amphibShips = shipsData.filter(s => s.ship_type !== 'CVN');
            
            // CVN trace (Carriers)
            const cvnTrace = {{
                type: 'scattergeo',
                mode: 'markers+text',
                lat: cvnShips.map(s => s.lat),
                lon: cvnShips.map(s => s.lon),
                text: cvnShips.map(s => s.hull),
                textposition: 'top center',
                textfont: {{
                    family: 'Orbitron, sans-serif',
                    size: 10,
                    color: '#ff6b6b'
                }},
                hoverinfo: 'text',
                hovertext: cvnShips.map(s => `<b>${{s.hull}}</b><br>${{s.name}}<br>${{s.location}}`),
                marker: {{
                    size: 14,
                    color: '#ff6b6b',
                    symbol: 'diamond',
                    line: {{
                        width: 2,
                        color: '#ff3333'
                    }}
                }},
                name: 'Carriers (CVN)',
                customdata: cvnShips
            }};
            
            // Amphib trace
            const amphibTrace = {{
                type: 'scattergeo',
                mode: 'markers+text',
                lat: amphibShips.map(s => s.lat),
                lon: amphibShips.map(s => s.lon),
                text: amphibShips.map(s => s.hull),
                textposition: 'top center',
                textfont: {{
                    family: 'Orbitron, sans-serif',
                    size: 9,
                    color: '#4ecdc4'
                }},
                hoverinfo: 'text',
                hovertext: amphibShips.map(s => `<b>${{s.hull}}</b><br>${{s.name}}<br>${{s.location}}`),
                marker: {{
                    size: 11,
                    color: '#4ecdc4',
                    symbol: 'circle',
                    line: {{
                        width: 2,
                        color: '#2eb8b0'
                    }}
                }},
                name: 'Amphibs (LHA/LHD)',
                customdata: amphibShips
            }};
            
            const layout = {{
                geo: {{
                    projection: {{
                        type: 'orthographic',
                        rotation: {{
                            lon: -100,
                            lat: 20
                        }}
                    }},
                    showland: true,
                    landcolor: '#1a1a2e',
                    showocean: true,
                    oceancolor: '#0a0a0f',
                    showlakes: false,
                    showrivers: false,
                    showcountries: true,
                    countrycolor: '#2a2a4e',
                    countrywidth: 0.5,
                    showcoastlines: true,
                    coastlinecolor: '#00ff88',
                    coastlinewidth: 1,
                    showframe: false,
                    bgcolor: 'rgba(0,0,0,0)',
                    lonaxis: {{
                        showgrid: true,
                        gridcolor: 'rgba(0, 255, 136, 0.1)',
                        gridwidth: 0.5
                    }},
                    lataxis: {{
                        showgrid: true,
                        gridcolor: 'rgba(0, 255, 136, 0.1)',
                        gridwidth: 0.5
                    }}
                }},
                paper_bgcolor: 'rgba(0,0,0,0)',
                plot_bgcolor: 'rgba(0,0,0,0)',
                margin: {{ t: 0, b: 0, l: 0, r: 0 }},
                showlegend: true,
                legend: {{
                    x: 0.02,
                    y: 0.98,
                    bgcolor: 'rgba(10, 10, 15, 0.8)',
                    bordercolor: '#00ff88',
                    borderwidth: 1,
                    font: {{
                        family: 'Share Tech Mono, monospace',
                        size: 11,
                        color: '#00ff88'
                    }}
                }},
                dragmode: 'pan'
            }};
            
            const config = {{
                responsive: true,
                displayModeBar: true,
                modeBarButtonsToRemove: ['toImage', 'sendDataToCloud'],
                displaylogo: false,
                scrollZoom: true
            }};
            
            Plotly.newPlot('globe', [cvnTrace, amphibTrace], layout, config);
            
            // Click handler for globe
            document.getElementById('globe').on('plotly_click', function(data) {{
                if (data.points && data.points.length > 0) {{
                    const point = data.points[0];
                    const ship = point.customdata;
                    if (ship) {{
                        showDetail(ship);
                        highlightShipCard(ship.hull);
                    }}
                }}
            }});
        }}
        
        // Render ship list
        function renderShipList() {{
            const container = document.getElementById('shipList');
            const filtered = activeFilter === 'all' 
                ? shipsData 
                : shipsData.filter(s => s.ship_type === activeFilter);
            
            container.innerHTML = filtered.map(ship => {{
                const typeClass = ship.ship_type === 'CVN' ? 'cvn' : 'amphib';
                return `
                    <div class="ship-card ${{typeClass}}" data-hull="${{ship.hull}}" onclick="selectShip('${{ship.hull}}')">
                        <div class="ship-header">
                            <span class="ship-hull ${{typeClass}}">${{ship.hull}}</span>
                            <span class="ship-type">${{ship.ship_type}}</span>
                        </div>
                        <div class="ship-name">${{ship.name}}</div>
                        <div class="ship-location">${{ship.location}}</div>
                        <div class="ship-date">As of ${{ship.date}}</div>
                    </div>
                `;
            }}).join('');
        }}
        
        // Select ship
        function selectShip(hull) {{
            const ship = shipsData.find(s => s.hull === hull);
            if (ship) {{
                showDetail(ship);
                highlightShipCard(hull);
                // Rotate globe to ship location
                Plotly.relayout('globe', {{
                    'geo.projection.rotation.lon': ship.lon,
                    'geo.projection.rotation.lat': ship.lat > 60 ? 60 : (ship.lat < -60 ? -60 : ship.lat)
                }});
            }}
        }}
        
        // Highlight ship card
        function highlightShipCard(hull) {{
            document.querySelectorAll('.ship-card').forEach(card => {{
                card.classList.remove('active');
            }});
            const card = document.querySelector(`.ship-card[data-hull="${{hull}}"]`);
            if (card) {{
                card.classList.add('active');
                card.scrollIntoView({{ behavior: 'smooth', block: 'center' }});
            }}
        }}
        
        // Show detail panel
        function showDetail(ship) {{
            selectedShip = ship;
            document.getElementById('detailPanel').classList.remove('hidden');
            document.getElementById('detailTitle').textContent = `${{ship.hull}} - ${{ship.name}}`;
            document.getElementById('detailClass').textContent = `${{ship.ship_class}}-class ${{ship.ship_type}}`;
            document.getElementById('detailLocation').textContent = ship.location;
            document.getElementById('detailRegion').textContent = ship.region;
            document.getElementById('detailDate').textContent = ship.date;
            document.getElementById('detailStatus').textContent = ship.status;
        }}
        
        // Close detail panel
        function closeDetail() {{
            document.getElementById('detailPanel').classList.add('hidden');
            document.querySelectorAll('.ship-card').forEach(card => {{
                card.classList.remove('active');
            }});
            selectedShip = null;
        }}
        
        // Filter handlers
        document.querySelectorAll('.filter-btn').forEach(btn => {{
            btn.addEventListener('click', function() {{
                document.querySelectorAll('.filter-btn').forEach(b => b.classList.remove('active'));
                this.classList.add('active');
                activeFilter = this.dataset.filter;
                renderShipList();
            }});
        }});
        
        // Initialize
        initGlobe();
        renderShipList();
    </script>
</body>
</html>
'''
    
    return html


# ==============================================================================
# MAIN EXECUTION
# ==============================================================================

# Scrape fleet data
fleet_data = scrape_fleet()

# Generate HTML
html_content = generate_globe_html(fleet_data)

# Save to file
output_file = "fleet_tracker_globe.html"
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"\n✓ HTML file saved: {output_file}")
print("  Download this file and open in any browser to share!\n")

# Display inline in Colab
display(HTML(html_content))

In [ ]:
# Cell 3: Download the HTML file (Run this to get download link in Colab)
from google.colab import files
files.download('fleet_tracker_globe.html')